In [ ]:
from pathlib import Path
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt
from pyometiff import OMETIFFWriter
import webknossos as wk

import dask.array as da

import pandas as pd
from skimage.measure import label, regionprops_table
from skimage.color import label2rgb

from webknossos import BoundingBox
import matplotlib.pyplot as plt

from webknossos_utils import Pixel_size, Annotation
import napari


In [ ]:

import io
use_napari = False
if use_napari:
    viewer = napari.Viewer()


In [ ]:

# this comes from the WEBKNOSSOS website, under the user settings
AUTH_TOKEN = "1mng65J7d-5IVFmfJoF4rw" # from gisela
WK_TIMEOUT="3600" # in seconds
ORG_ID = "83d574429f8bc523" # gisela's webknossos

#sample1   = Annotation("6644c04d0100004a01fa11af", "deprecated","60_2_5R")
#sample2    = Annotation("664316880100008a049e890e", "deprecated","60_2_3L")
id_1 = "6644c04d0100004a01fa11af"
id_2 = "664316880100008a049e890e"
id_3 = "664606440100005102550210"

#666c4ae70100002b015e2344
# the dataset url comes from the WEBKNOSSOS website, open the image of interest from the dashboard and check
# I removed the view information

ANNOTATION_ID = id_1

with wk.webknossos_context(token=AUTH_TOKEN):
    annotations = wk.Annotation.open_as_remote_dataset(annotation_id_or_url=ANNOTATION_ID)
    lbl_layers = annotations.get_segmentation_layers()

    DATASET_NAME = annotations._properties.id['name']
    ds = wk.Dataset.open_remote(dataset_name_or_url=DATASET_NAME, organization_id=ORG_ID)
    img_layer = ds.get_color_layers()
    assert len(img_layer) == 1, "more than an image, this is unexpected for this project"
    img_layer = img_layer[0]

    #dataset = wk.Dataset.open_remote(DATASET_URL)
    #img_data = dataset.get_color_layer().get_finest_mag().read()
    


In [ ]:
voxel_size = ds.voxel_size
print(img_layer)
print(f'image size in pixels {img_layer.bounding_box.size.to_np()}')
print(f'voxel size in nm: {ds.voxel_size}')


In [ ]:

mag_list = list(img_layer.mags.keys())
print(mag_list)
MAG = mag_list[3]
pSize = Pixel_size(voxel_size[0] * MAG.x, voxel_size[1] * MAG.y, voxel_size[2] * MAG.z, MAG=MAG, unit="nm")
print(pSize)
with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    img_data = img_layer.get_mag(pSize.MAG).read()
print(img_data.shape)
img_dask = da.from_array(np.swapaxes(img_data,-1,-3), chunks=(1,1,512,512))
img_dask

output_fpath = Path.cwd().joinpath("gisela_ds_" + img_layer.name + "_mag_" + str(pSize.MAG) + ".ome.tiff")
print(output_fpath)

assert pSize.unit == "nm", "should be nm for this to work"
metadata_dict = {
    "PhysicalSizeX" : str(pSize.x/1000),
    "PhysicalSizeXUnit" : "µm",
    "PhysicalSizeY" : str(pSize.y/1000),
    "PhysicalSizeYUnit" : "µm",
    "PhysicalSizeZ" : str(pSize.z/1000),
    "PhysicalSizeZUnit" : "µm",
    "Channels" : {
        "SEM" : {
            "Name" : "BSD",
            "SamplesPerPixel": 1,
        },
    }
}

# a string describing the dimension ordering
dimension_order = "CZYX"

# writer = OMETIFFWriter(
#     fpath=output_fpath,
#     dimension_order=dimension_order,
#     array=img_dask,
#     metadata=metadata_dict,
#     explicit_tiffdata=False)

# writer.write()


In [ ]:

if use_napari:
    viewer.add_image(data=img_dask, channel_axis=0, name=img_layer.name)


In [ ]:

for lbl in lbl_layers:
    print(lbl)
    print(lbl.name)
    print(lbl.bounding_box.size.to_np())


In [ ]:
for i, lbl in enumerate(lbl_layers):
    if lbl.name == "Myelin":
        myelin_idx = i
    elif lbl.name == "Axon":
        axon_idx = i
    elif lbl.name == "Mitochondria":
        mito_idx = i
    elif lbl.name == "Dystrophic_myelin":
        dystrophic_idx = i

print(myelin_idx)
print(axon_idx)


In [ ]:
with wk.webknossos_context(token=AUTH_TOKEN):
    lbl_data = lbl_layers[myelin_idx].get_mag(pSize.MAG).read()
unique_lbls = np.unique(lbl_data)
print(unique_lbls)


if np.max(unique_lbls) < 512:
    lbl_data = lbl_data.astype(np.uint8)

lbl_dask = da.from_array(np.swapaxes(lbl_data,-1,-3), chunks=(1,5,512,512))
lbl_dask

output_fpath = Path.cwd().joinpath("gisela_ds_" + img_layer.name + "_mag_" + str(pSize.MAG) + "_lbl_Myelin.ome.tiff")
print(output_fpath)

assert pSize.unit == "nm", "should be nm for this to work"
metadata_dict = {
    "PhysicalSizeX" : str(pSize.x/1000),
    "PhysicalSizeXUnit" : "µm",
    "PhysicalSizeY" : str(pSize.y/1000),
    "PhysicalSizeYUnit" : "µm",
    "PhysicalSizeZ" : str(pSize.z/1000),
    "PhysicalSizeZUnit" : "µm",
    "Channels" : {
        "SEM" : {
            "Name" : "Myelin",
            "SamplesPerPixel": 1,
        },
    }
}

# a string describing the dimension ordering
dimension_order = "CZYX"

segmentation = np.nonzero(lbl_dask[0][0])

bbox = 0, 0, 0, 0
#if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
y_min = int(np.min(segmentation[1]))
y_max = int(np.max(segmentation[1]))
x_min = int(np.min(segmentation[0]))
x_max = int(np.max(segmentation[0]))

large_bbox = x_min, x_max, y_min, y_max


In [ ]:
aoi = lbl_data[0][y_min:y_max,x_min:x_max]
#plt.imshow(lbl_data[0])
fig = plt.figure(figsize=(100, 100)) 
plt.imshow(aoi)


In [ ]:
import myelin_morphometrics as morpho
from skimage.morphology import convex_hull_image
from skimage.segmentation import active_contour
from skimage.measure import find_contours
from webknossos_utils import skibbox2wkbbox
from scipy import ndimage

properties = ['label', 'bbox']
reg_table = regionprops_table(label_image=lbl_dask[0,0,:,:],
                          properties=properties)
reg_table = pd.DataFrame(reg_table)

#plt.imshow(lbl_dask[0][0])

fig = plt.figure(figsize=(100, 100)) 
 


for index, row in reg_table.iterrows():
    #print(row)
    #if index < 50:
    #    continue 

    obj_idx = row['label']

    bbox = skibbox2wkbbox(row.to_dict(), pSize)
    #img_data = img_layer.get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)
    img_data = img_layer.get_finest_mag().read(absolute_offset=(x_min,y_max,0), size=(x_max-x_min,y_max-y_min,1))
    #img_data = img_layer.get_finest_mag().read()

    myelin_lbl = lbl_layers[myelin_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()

    myelin_bw = morpho.get_BW_from_lbl(myelin_lbl, obj_idx)
    # clean myelin label map, in case other neurons are close by
    myelin_lbl[np.logical_not(myelin_bw)] = 0

    #properties = ['label', 'area','eccentricity','solidity','intensity_mean',
    #            'axis_minor_length','axis_major_length','feret_diameter_max']
    # obj_table = regionprops_table(label_image=myelin_lbl,
    #                                 intensity_image=img_data.squeeze(),
    #                                 properties=properties)
    # obj_table = pd.DataFrame(obj_table)
    # obj_table.rename(columns={'area': 'myelin_area', 
    #                     'eccentricity': 'myelin_eccentricity',
    #                     'solidity': 'myelin_solidity',
    #                     'intensity_mean': 'myelin_intensity_mean',
    #                     'axis_minor_length': 'myelin_axis_minor_length',
    #                     'axis_major_length': 'myelin_axis_major_length',
    #                     'feret_diameter_max': 'myelin_feret_diameter_max'}, inplace=True)

    # assert len(obj_table.axes[0]) == 1, "we expected to have a single object"
    
    # Create the padded myelin_bw image to avoid edge effects in the contours
    myelin_bw_fill = ndimage.binary_fill_holes(myelin_bw)

    com = ndimage.center_of_mass(myelin_bw_fill)
    expected_filled = myelin_bw_fill[int(com[0]), int(com[1])]

    if not expected_filled:
        myelin_bw_ch = convex_hull_image(myelin_bw_fill)
        myelin_bw_invert = np.invert(myelin_bw_fill)
        myelin_bw_solid = np.logical_and(myelin_bw_invert,myelin_bw_ch)
        myelin_bw_fill = np.logical_or(myelin_bw_fill,myelin_bw_solid)


    #myelin_bw_solid = ndimage.binary_fill_holes(myelin_bw_solid)

    #fig.add_subplot(10, 21, index+1)
    #snake = active_contour(myelin_bw_fill, (100,100), w_edge=0, w_line=1)
    #clear_output(wait=True)
    plt.imshow(myelin_bw_fill)
    #plt.imshow(img_data[0])

#    ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)

#    ax.plot(tmp[:, 0], tmp[:, 1], linewidth=2)

 #   ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)
    #ax.set_title(f'Image {index}')

    #fig.canvas.draw()
    # Display the plot (necessary for Jupyter Notebooks)
    plt.show()


    #c = ax.contour(myelin_bw_fill)
    print(index)
#    plt.imshow(myelin_bw_solid)
    #plt.imshow(myelin_bw)
    #fig.canvas.draw()
    #plt.imshow(snake)
    #plt.show()


#    if index >= 50:
#        break
    # create the "axon area", this is the internal area of the myelin region
    #auto_axon_region = np.logical_xor(myelin_bw_fill,myelin_bw)
    #if np.sum(auto_axon_region)<100:
        #warnings.warn("The axon was most probably not closed.")
    #   myelin_bw_fill = convex_hull_image(myelin_bw_fill)
plt.show()

In [ ]:


from skimage.draw import rectangle, rectangle_perimeter
import matplotlib.pyplot as plt

#rr,cc = rectangle_perimeter((x_min,y_min), end=(x_max,y_max), shape=lbl_dask[0][0].shape)
#img = np.ndarray(lbl_dask[0][0].shape)
#img[rr,cc] = 10

#img





img_data = img_layer.get_mag(pSize.MAG).read()
img_data = np.swapaxes(img_data,-2,-3)

fig, ax = plt.subplots(figsize=(15, 15))
#ax.imshow(random.rand(8, 90), interpolation='nearest')
#color1 = label2rgb(lbl_data.squeeze().T, image=img_data.squeeze().T, bg_label=0)
aoi = lbl_dask[0][0][x_min:x_max, y_min:y_max:]
aoii = img_data[:y_min:y_max, x_min:x_max:] 
#plt.imshow(lbl_dask[0][0], cmap=plt.colormaps["gist_rainbow"])
ax.imshow(aoii[0])
ax.imshow(aoi, cmap=plt.colormaps["tab20c"], interpolation="nearest",alpha=0.5)
#plt.imshow(img,alpha=0.5)

# writer = OMETIFFWriter(
#     fpath=output_fpath,
#     dimension_order=dimension_order,
#     array=lbl_dask,
#     metadata=metadata_dict,
#     explicit_tiffdata=False)

# writer.write()


In [ ]:
if use_napari:
    viewer.add_labels(lbl_dask, name="myelin")
properties = ['label', 'bbox']
reg_table = regionprops_table(label_image=lbl_dask[0,0,:,:],
                          properties=properties)
reg_table = pd.DataFrame(reg_table)
#reg_table.sample(201)
from webknossos_utils import skibbox2wkbbox
bbox = skibbox2wkbbox(reg_table.iloc[56].to_dict(), pSize)
bbox

with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    img_data = img_layer.get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)
    #img_data = img_layer.get_mag(pSize.MAG).read(absolute_offset=bbox.topleft, size=bbox.size)
    lbl_data = lbl_layers[myelin_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)
    #lbl_data = lbl_layers[annotation_idx].get_mag(pSize.MAG).read(absolute_offset=bbox.topleft, size=bbox.size)
print(img_data.shape)

color1 = label2rgb(lbl_data.squeeze().T, image=img_data.squeeze().T, bg_label=0)
plt.imshow(color1)


In [ ]:

from time import sleep
from IPython.display import clear_output
from skimage.morphology import convex_hull_image
from skimage.measure import find_contours
from scipy import ndimage

import os
import myelin_morphometrics as morpho
import warnings
from scipy.ndimage import center_of_mass

def bbox_from_BWimg(BW):
    # convex hull calculation including smoothing to avoid pixelation effects
    chull_img = convex_hull_image(BW)
    chull_contour = find_contours(chull_img)
    assert len(chull_contour)==1, "we only expect one contour"

    convex_hull = morpho.smoothBoundary(chull_contour[0])
    min_box_corners, min_box_edges, rot_mat = morpho.minBoundingBox(convex_hull, 'width')

    min_box_corners = min_box_corners - np.mean(min_box_corners, axis=0)
    #   move the box to the same reference as the inpurt convex hull
    c_mass = center_of_mass(chull_img, labels=None, index=None)
    min_box_corners = min_box_corners + c_mass


    return min_box_corners, min_box_edges, rot_mat

def get_box_with_rotation(bw_img, rot_mat):
    chull_img = convex_hull_image(bw_img)
    chull_contour = find_contours(chull_img)
    assert len(chull_contour)==1, "we only expect one contour"
    convex_hull = morpho.smoothBoundary(chull_contour[0])
    r_hull = morpho.rotate_points(convex_hull, rot_mat)
    box_dim = morpho.bbox_dimenssions(r_hull)

    #print(box_dim)

    min_bbox_points = np.array([
        [0, 0],
        [box_dim[0], 0],
        [box_dim[0], box_dim[1]],
        [0, box_dim[1]]
    ])

    #   rotate the box
    min_bbox_corners = morpho.rotate_points(min_bbox_points, rot_mat.T)
    #   move the box to 0,0
    min_bbox_corners = min_bbox_corners - np.mean(min_bbox_corners, axis=0)
    #   move the box to the same reference as the inpurt convex hull
    c_mass = center_of_mass(chull_img, labels=None, index=None)
    min_bbox_corners = min_bbox_corners + c_mass #((np.max(convex_hull, axis=0)-np.min(convex_hull, axis=0))/2)+np.min(convex_hull, axis=0)

    return min_bbox_corners, box_dim
out_folder = Path("./output/"+DATASET_NAME)
out_folder

# Check if the folder exists
if not out_folder.exists():
    # If the folder doesn't exist, create it
    
    os.makedirs(out_folder)
    print(f"Folder '{out_folder}' created successfully.")
else:
    print(f"Folder '{out_folder}' already exists.")
from skimage.morphology import remove_small_objects, isotropic_erosion
total_table = []
pad_size = 5
fig = plt.figure()) 
with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    for index, row in reg_table.iterrows():
        print(row)
        obj_idx = row['label']

        bbox = skibbox2wkbbox(row.to_dict(), pSize)
        img_data = img_layer.get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)

        myelin_lbl = lbl_layers[myelin_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        axon_lbl   = lbl_layers[axon_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        mito_lbl   = lbl_layers[mito_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        dyst_lbl   = lbl_layers[dystrophic_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()


        # get area of the iamge identified as myelin, this is my main ROI which defines behaviour of all others.
        myelin_bw = morpho.get_BW_from_lbl(myelin_lbl, obj_idx)
        # clean myelin label map, in case other neurons are close by
        myelin_lbl[np.logical_not(myelin_bw)] = 0

        properties = ['label', 'area','eccentricity','solidity','intensity_mean',
                      'axis_minor_length','axis_major_length','feret_diameter_max']
        obj_table = regionprops_table(label_image=myelin_lbl,
                                      intensity_image=img_data.squeeze(),
                                      properties=properties)
        obj_table = pd.DataFrame(obj_table)
        obj_table.rename(columns={'area': 'myelin_area', 
                          'eccentricity': 'myelin_eccentricity',
                          'solidity': 'myelin_solidity',
                          'intensity_mean': 'myelin_intensity_mean',
                          'axis_minor_length': 'myelin_axis_minor_length',
                          'axis_major_length': 'myelin_axis_major_length',
                          'feret_diameter_max': 'myelin_feret_diameter_max'}, inplace=True)

        assert len(obj_table.axes[0]) == 1, "we expected to have a single object"
        
        # Create the padded myelin_bw image to avoid edge effects in the contours
        myelin_bw_fill = ndimage.binary_fill_holes(myelin_bw)
        # create the "axon area", this is the internal area of the myelin region
        auto_axon_region = np.logical_xor(myelin_bw_fill,myelin_bw)
        if np.sum(auto_axon_region)<100:
            warnings.warn("The axon was most probably not closed.")
            myelin_bw_fill = convex_hull_image(myelin_bw_fill)
            auto_axon_region = np.logical_xor(myelin_bw_fill,myelin_bw)
            auto_axon_region = isotropic_erosion(auto_axon_region, 3)
            auto_axon_region = remove_small_objects(auto_axon_region, min_size=500)
            assert np.sum(auto_axon_region)>500, "unexpected problems with the auto axon region"



        myelin_filled_area = np.sum(myelin_bw_fill)
        obj_table['myelin_filled_area'] =myelin_filled_area


        myelin_bw_pad = np.pad(myelin_bw_fill, ((pad_size, pad_size), (pad_size, pad_size),), mode='constant', constant_values=0)
        # convex hull calculation including smoothing to avoid pixelation effects
        myelin_min_box_corners, myelin_min_box_edges, rot_mat = bbox_from_BWimg(myelin_bw_pad)
        # using the bbox as feret calculator and AR
        obj_table['myelin_feret_max'] = np.max(myelin_min_box_edges)
        obj_table['myelin_feret_min'] = np.min(myelin_min_box_edges)
        obj_table['myelin_AR'] = np.max(myelin_min_box_edges)/np.min(myelin_min_box_edges)
        # calculating myelin median width
        obj_table['myelin_width'] = morpho.get_width(myelin_bw)


        auto_axon_area = np.sum(auto_axon_region)
        obj_table['myelin_hole_area'] =auto_axon_area

        max_axon_bbox, max_axon_size = get_box_with_rotation(auto_axon_region, rot_mat.T)

        myelin_width_min_feret = np.min(myelin_min_box_edges)-np.min(max_axon_size)
        myelin_width_max_feret = np.max(myelin_min_box_edges)-np.max(max_axon_size)
        obj_table['myelin_width_min_feret_direction'] = myelin_width_min_feret/2
        obj_table['myelin_width_max_feret_direction'] = myelin_width_max_feret/2

        # cleaning axon label map
        axon_bw = axon_lbl>0
        axon_bw = np.logical_and(myelin_bw_fill, axon_bw)
        user_def_axon = np.any(axon_bw)

        if not(user_def_axon):
            print('no user defined axon, we will create one based on the myelin')
            axon_bw = auto_axon_region 
        assert np.any(axon_bw), "The axon label map is empty: does not contain any True values"
        
        # I do not expect holes in the axon image
        axon_bw = ndimage.binary_fill_holes(axon_bw)
        axon_bw_pad = np.pad(axon_bw, ((pad_size, pad_size), (pad_size, pad_size),), mode='constant', constant_values=0)

        # convex hull calculation including smoothing to avoid pixelation effects
        axon_min_box_corners, axon_min_box_edges, _ = bbox_from_BWimg(axon_bw_pad)

        # using the bbox as feret calculator and AR
        obj_table['axon_area'] = np.sum(axon_bw_pad)
        obj_table['axon_feret_max'] = np.max(axon_min_box_edges)
        obj_table['axon_feret_min'] = np.min(axon_min_box_edges)
        obj_table['axon_AR'] = np.max(axon_min_box_edges)/np.min(axon_min_box_edges)

        #check the mito labels
        mito_bw = mito_lbl>0
        mito_bw = np.logical_and(myelin_bw_fill, mito_bw)
        user_def_mito = np.any(mito_bw)


        if not(user_def_mito):
            print('no mito in the neuron')
            mito_number = 0
            mito_total_area = 0
        else:
            mito_lbl = label(mito_bw)
            mito_number = np.max(mito_lbl)
            mito_total_area = np.sum(mito_bw)

        obj_table['mito_total_area'] = mito_total_area
        obj_table['mito_number'] = mito_number


        # cleaning the distrophic label:
        # cleaning axon label map
        dyst_bw = dyst_lbl>0
        dyst_bw = np.logical_and(myelin_bw_fill, dyst_bw)
        is_dystrophic = np.any(dyst_bw)
        obj_table['is_dystrophic'] = is_dystrophic
        # for later plot
        axon_lbl[np.logical_not(axon_bw)] = 0
        axon_lbl[axon_bw] = obj_table['label'] + 1

        mito_lbl[np.logical_not(mito_bw)] = 0
        mito_lbl[mito_bw] = obj_table['label'] + 2

        dyst_lbl[dyst_lbl>0] = obj_table['label'] + 3

        color = label2rgb(myelin_lbl.T + axon_lbl.T + mito_lbl.T + dyst_lbl.T, image=img_data.squeeze().T, bg_label=0)

        # Plot the image
        clear_output(wait=True)
        # Create a figure and axis for plotting
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.imshow(color)
        tmp = np.vstack((myelin_min_box_corners, myelin_min_box_corners[0,:]))
        ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)

        tmp = np.vstack((max_axon_bbox, max_axon_bbox[0,:]))
        ax.plot(tmp[:, 0], tmp[:, 1], linewidth=2)

        tmp = np.vstack((axon_min_box_corners, axon_min_box_corners[0,:]))
        ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)



        ax.set_title(f'Image {obj_idx}')

        fig.canvas.draw()
        # Display the plot (necessary for Jupyter Notebooks)
        png_name = f'myelin_label_{obj_idx}.png'
        plt.savefig(out_folder.joinpath(png_name))
        plt.show()

        if isinstance(total_table, pd.DataFrame):
            # then I concatenate
            total_table = pd.concat([total_table, obj_table], axis=0)
        else:
            total_table = obj_table

        print(f"done for index: {obj_idx}")

        #if index>=1:
        #    break
# Write the DataFrame to an Excel file
xlsx_name = DATASET_NAME + "_morphometrics.xlsx"
total_table.to_excel(out_folder.joinpath(xlsx_name), index=False, engine='xlsxwriter')
total_table

In [ ]:

writer.write()
if use_napari:
    viewer.add_image(data=img_dask, channel_axis=0, name=img_layer.name)
for lbl in lbl_layers:
    print(lbl)
    print(lbl.name)
    print(lbl.bounding_box.size.to_np())
for i, lbl in enumerate(lbl_layers):
    if lbl.name == "Myelin":
        myelin_idx = i
    elif lbl.name == "Axon":
        axon_idx = i
    elif lbl.name == "Mitochondria":
        mito_idx = i
    elif lbl.name == "Dystrophic_myelin":
        dystrophic_idx = i

print(myelin_idx)
print(axon_idx)
with wk.webknossos_context(token=AUTH_TOKEN):
    lbl_data = lbl_layers[myelin_idx].get_mag(pSize.MAG).read()
unique_lbls = np.unique(lbl_data)
print(unique_lbls)


if np.max(unique_lbls) < 512:
    lbl_data = lbl_data.astype(np.uint8)

lbl_dask = da.from_array(np.swapaxes(lbl_data,-1,-3), chunks=(1,5,512,512))
lbl_dask

output_fpath = Path.cwd().joinpath("gisela_ds_" + img_layer.name + "_mag_" + str(pSize.MAG) + "_lbl_Myelin.ome.tiff")
print(output_fpath)

assert pSize.unit == "nm", "should be nm for this to work"
metadata_dict = {
    "PhysicalSizeX" : str(pSize.x/1000),
    "PhysicalSizeXUnit" : "µm",
    "PhysicalSizeY" : str(pSize.y/1000),
    "PhysicalSizeYUnit" : "µm",
    "PhysicalSizeZ" : str(pSize.z/1000),
    "PhysicalSizeZUnit" : "µm",
    "Channels" : {
        "SEM" : {
            "Name" : "Myelin",
            "SamplesPerPixel": 1,
        },
    }
}

# a string describing the dimension ordering
dimension_order = "CZYX"

writer = OMETIFFWriter(
    fpath=output_fpath,
    dimension_order=dimension_order,
    array=lbl_dask,
    metadata=metadata_dict,
    explicit_tiffdata=False)

writer.write()
if use_napari:
    viewer.add_labels(lbl_dask, name="myelin")
properties = ['label', 'bbox']
reg_table = regionprops_table(label_image=lbl_dask[0,0,:,:],
                          properties=properties)
reg_table = pd.DataFrame(reg_table)
reg_table.sample(5)
from webknossos_utils import skibbox2wkbbox
bbox = skibbox2wkbbox(reg_table.iloc[0].to_dict(), pSize)
bbox

with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    img_data = img_layer.get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)
    #img_data = img_layer.get_mag(pSize.MAG).read(absolute_offset=bbox.topleft, size=bbox.size)
    lbl_data = lbl_layers[myelin_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)
    #lbl_data = lbl_layers[annotation_idx].get_mag(pSize.MAG).read(absolute_offset=bbox.topleft, size=bbox.size)
print(img_data.shape)

color1 = label2rgb(lbl_data.squeeze().T, image=img_data.squeeze().T, bg_label=0)
plt.imshow(color1)

from time import sleep
from IPython.display import clear_output
from skimage.morphology import convex_hull_image
from skimage.measure import find_contours
from scipy import ndimage

import myelin_morphometrics as morpho
import warnings
from scipy.ndimage import center_of_mass

def bbox_from_BWimg(BW):
    # convex hull calculation including smoothing to avoid pixelation effects
    chull_img = convex_hull_image(BW)
    chull_contour = find_contours(chull_img)
    assert len(chull_contour)==1, "we only expect one contour"

    convex_hull = morpho.smoothBoundary(chull_contour[0])
    min_box_corners, min_box_edges, rot_mat = morpho.minBoundingBox(convex_hull, 'width')

    min_box_corners = min_box_corners - np.mean(min_box_corners, axis=0)
    #   move the box to the same reference as the inpurt convex hull
    c_mass = center_of_mass(chull_img, labels=None, index=None)
    min_box_corners = min_box_corners + c_mass


    return min_box_corners, min_box_edges, rot_mat

def get_box_with_rotation(bw_img, rot_mat):
    chull_img = convex_hull_image(bw_img)
    chull_contour = find_contours(chull_img)
    assert len(chull_contour)==1, "we only expect one contour"
    convex_hull = morpho.smoothBoundary(chull_contour[0])
    r_hull = morpho.rotate_points(convex_hull, rot_mat)
    box_dim = morpho.bbox_dimenssions(r_hull)

    #print(box_dim)

    min_bbox_points = np.array([
        [0, 0],
        [box_dim[0], 0],
        [box_dim[0], box_dim[1]],
        [0, box_dim[1]]
    ])

    #   rotate the box
    min_bbox_corners = morpho.rotate_points(min_bbox_points, rot_mat.T)
    #   move the box to 0,0
    min_bbox_corners = min_bbox_corners - np.mean(min_bbox_corners, axis=0)
    #   move the box to the same reference as the inpurt convex hull
    c_mass = center_of_mass(chull_img, labels=None, index=None)
    min_bbox_corners = min_bbox_corners + c_mass #((np.max(convex_hull, axis=0)-np.min(convex_hull, axis=0))/2)+np.min(convex_hull, axis=0)

    return min_bbox_corners, box_dim
out_folder = Path("./output/"+DATASET_NAME)
out_folder

# Check if the folder exists
if not out_folder.exists():
    # If the folder doesn't exist, create it
    out_folder.mkdir()
    print(f"Folder '{out_folder}' created successfully.")
else:
    print(f"Folder '{out_folder}' already exists.")
from skimage.morphology import remove_small_objects, isotropic_erosion
total_table = []
pad_size = 5
with wk.webknossos_context(token=AUTH_TOKEN, timeout=WK_TIMEOUT):
    for index, row in reg_table.iterrows():
        print(row)
        obj_idx = row['label']

        bbox = skibbox2wkbbox(row.to_dict(), pSize)
        img_data = img_layer.get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size)

        myelin_lbl = lbl_layers[myelin_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        axon_lbl   = lbl_layers[axon_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        mito_lbl   = lbl_layers[mito_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()
        dyst_lbl   = lbl_layers[dystrophic_idx].get_finest_mag().read(absolute_offset=bbox.topleft, size=bbox.size).squeeze()


        # get area of the iamge identified as myelin, this is my main ROI which defines behaviour of all others.
        myelin_bw = morpho.get_BW_from_lbl(myelin_lbl, obj_idx)
        # clean myelin label map, in case other neurons are close by
        myelin_lbl[np.logical_not(myelin_bw)] = 0

        properties = ['label', 'area','eccentricity','solidity','intensity_mean',
                      'axis_minor_length','axis_major_length','feret_diameter_max']
        obj_table = regionprops_table(label_image=myelin_lbl,
                                      intensity_image=img_data.squeeze(),
                                      properties=properties)
        obj_table = pd.DataFrame(obj_table)
        obj_table.rename(columns={'area': 'myelin_area', 
                          'eccentricity': 'myelin_eccentricity',
                          'solidity': 'myelin_solidity',
                          'intensity_mean': 'myelin_intensity_mean',
                          'axis_minor_length': 'myelin_axis_minor_length',
                          'axis_major_length': 'myelin_axis_major_length',
                          'feret_diameter_max': 'myelin_feret_diameter_max'}, inplace=True)

        assert len(obj_table.axes[0]) == 1, "we expected to have a single object"
        
        # Create the padded myelin_bw image to avoid edge effects in the contours
        myelin_bw_fill = ndimage.binary_fill_holes(myelin_bw)
        # create the "axon area", this is the internal area of the myelin region
        auto_axon_region = np.logical_xor(myelin_bw_fill,myelin_bw)
        if np.sum(auto_axon_region)<100:
            warnings.warn("The axon was most probably not closed.")
            myelin_bw_fill = convex_hull_image(myelin_bw_fill)
            auto_axon_region = np.logical_xor(myelin_bw_fill,myelin_bw)
            auto_axon_region = isotropic_erosion(auto_axon_region, 3)
            auto_axon_region = remove_small_objects(auto_axon_region, min_size=500)
            assert np.sum(auto_axon_region)>500, "unexpected problems with the auto axon region"



        myelin_filled_area = np.sum(myelin_bw_fill)
        obj_table['myelin_filled_area'] =myelin_filled_area


        myelin_bw_pad = np.pad(myelin_bw_fill, ((pad_size, pad_size), (pad_size, pad_size),), mode='constant', constant_values=0)
        # convex hull calculation including smoothing to avoid pixelation effects
        myelin_min_box_corners, myelin_min_box_edges, rot_mat = bbox_from_BWimg(myelin_bw_pad)
        # using the bbox as feret calculator and AR
        obj_table['myelin_feret_max'] = np.max(myelin_min_box_edges)
        obj_table['myelin_feret_min'] = np.min(myelin_min_box_edges)
        obj_table['myelin_AR'] = np.max(myelin_min_box_edges)/np.min(myelin_min_box_edges)
        # calculating myelin median width
        obj_table['myelin_width'] = morpho.get_width(myelin_bw)


        auto_axon_area = np.sum(auto_axon_region)
        obj_table['myelin_hole_area'] =auto_axon_area

        max_axon_bbox, max_axon_size = get_box_with_rotation(auto_axon_region, rot_mat.T)

        myelin_width_min_feret = np.min(myelin_min_box_edges)-np.min(max_axon_size)
        myelin_width_max_feret = np.max(myelin_min_box_edges)-np.max(max_axon_size)
        obj_table['myelin_width_min_feret_direction'] = myelin_width_min_feret/2
        obj_table['myelin_width_max_feret_direction'] = myelin_width_max_feret/2

        # cleaning axon label map
        axon_bw = axon_lbl>0
        axon_bw = np.logical_and(myelin_bw_fill, axon_bw)
        user_def_axon = np.any(axon_bw)

        if not(user_def_axon):
            print('no user defined axon, we will create one based on the myelin')
            axon_bw = auto_axon_region 
        assert np.any(axon_bw), "The axon label map is empty: does not contain any True values"
        
        # I do not expect holes in the axon image
        axon_bw = ndimage.binary_fill_holes(axon_bw)
        axon_bw_pad = np.pad(axon_bw, ((pad_size, pad_size), (pad_size, pad_size),), mode='constant', constant_values=0)

        # convex hull calculation including smoothing to avoid pixelation effects
        axon_min_box_corners, axon_min_box_edges, _ = bbox_from_BWimg(axon_bw_pad)

        # using the bbox as feret calculator and AR
        obj_table['axon_area'] = np.sum(axon_bw_pad)
        obj_table['axon_feret_max'] = np.max(axon_min_box_edges)
        obj_table['axon_feret_min'] = np.min(axon_min_box_edges)
        obj_table['axon_AR'] = np.max(axon_min_box_edges)/np.min(axon_min_box_edges)

        #check the mito labels
        mito_bw = mito_lbl>0
        mito_bw = np.logical_and(myelin_bw_fill, mito_bw)
        user_def_mito = np.any(mito_bw)


        if not(user_def_mito):
            print('no mito in the neuron')
            mito_number = 0
            mito_total_area = 0
        else:
            mito_lbl = label(mito_bw)
            mito_number = np.max(mito_lbl)
            mito_total_area = np.sum(mito_bw)

        obj_table['mito_total_area'] = mito_total_area
        obj_table['mito_number'] = mito_number


        # cleaning the distrophic label:
        # cleaning axon label map
        dyst_bw = dyst_lbl>0
        dyst_bw = np.logical_and(myelin_bw_fill, dyst_bw)
        is_dystrophic = np.any(dyst_bw)
        obj_table['is_dystrophic'] = is_dystrophic
        # for later plot
        axon_lbl[np.logical_not(axon_bw)] = 0
        axon_lbl[axon_bw] = obj_table['label'] + 1

        mito_lbl[np.logical_not(mito_bw)] = 0
        mito_lbl[mito_bw] = obj_table['label'] + 2

        dyst_lbl[dyst_lbl>0] = obj_table['label'] + 3

        color = label2rgb(myelin_lbl.T + axon_lbl.T + mito_lbl.T + dyst_lbl.T, image=img_data.squeeze().T, bg_label=0)

        # Plot the image
        clear_output(wait=True)
        # Create a figure and axis for plotting
        fig, ax = plt.subplots(figsize=(8, 8))

        ax.imshow(color)
        tmp = np.vstack((myelin_min_box_corners, myelin_min_box_corners[0,:]))
        ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)

        tmp = np.vstack((max_axon_bbox, max_axon_bbox[0,:]))
        ax.plot(tmp[:, 0], tmp[:, 1], linewidth=2)

        tmp = np.vstack((axon_min_box_corners, axon_min_box_corners[0,:]))
        ax.plot(tmp[:, 0]-pad_size, tmp[:, 1]-pad_size, linewidth=2)



        ax.set_title(f'Image {obj_idx}')

        fig.canvas.draw()
        # Display the plot (necessary for Jupyter Notebooks)
        png_name = f'myelin_label_{obj_idx}.png'
        plt.savefig(out_folder.joinpath(png_name))
        plt.show()

        if isinstance(total_table, pd.DataFrame):
            # then I concatenate
            total_table = pd.concat([total_table, obj_table], axis=0)
        else:
            total_table = obj_table

        print(f"done for index: {obj_idx}")

        #if index>=1:
        #    break
# Write the DataFrame to an Excel file
xlsx_name = DATASET_NAME + "_morphometrics.xlsx"
total_table.to_excel(out_folder.joinpath(xlsx_name), index=False, engine='xlsxwriter')
total_table